In [1]:
import pandas as pd
import warnings
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Concatenate, Attention
from tensorflow.keras.callbacks import EarlyStopping
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from tcn import TCN

warnings.filterwarnings('ignore')


2023-12-16 16:37:52.033583: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-16 16:37:52.077461: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-16 16:37:52.077496: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-16 16:37:52.077521: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-16 16:37:52.085571: I tensorflow/core/platform/cpu_feature_g

In [21]:
file_path = "myData2.parquet"
df = pd.read_parquet(file_path)


In [23]:
selected_features = ['timestamp_seconds',
                     'node_memory_Percpu_bytes', 
                     'node_context_switches_total', 
                     'surfsara_power_usage', 
                     'node_netstat_Tcp_InSegs', 
                     'node_netstat_Tcp_OutSegs', 
                     'node_network_transmit_packets_total-sum', 
                     'node_filesystem_size_bytes-sum', 
                     'node_filesystem_files-sum', 
                     'node_memory_MemFree_bytes', 
                     'node_netstat_Tcp_InErrs']


In [28]:
# Extract relevant columns
df_selected = df[['timestamp', 'state'] + selected_features].copy()

# Encode the target variable 'state' to binary (0 for "COMPLETED", 1 otherwise)
df_selected['target'] = (df_selected['state'] != 'COMPLETED').astype(int)

# Drop the original 'state' column
df_selected.drop('state', axis=1, inplace=True)

# Define time intervals
time_intervals = {'minute': '1T', 'hour': '1H', 'day': '1D'}

# Normalize selected features
scaler = MinMaxScaler()
df_selected[selected_features] = scaler.fit_transform(df_selected[selected_features])

# Set sequence length
sequence_length = 30


In [5]:
# Function to prepare data
def prepare_data(data, time_interval):
    data.set_index('timestamp', inplace=True)
    data_resampled = data.resample(time_interval).sum()
    data_resampled['target'] = data_resampled['target'].clip(upper=1)  # Clip values to 1
    return data_resampled

# Function to create sequences
def create_sequences(data, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        seq = data.iloc[i:i+sequence_length].values
        target = data.iloc[i+sequence_length]['target']
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)


In [6]:
# Function to make predictions on new data for the hybrid model
def predict_future_failures_hybrid(model, input_data_lstm, input_data_tcn, sequence_length, prediction_steps):
    predictions = []

    for _ in range(prediction_steps):
        # Make predictions for the next time step using both LSTM and TCN models
        prediction = model.predict([input_data_lstm.reshape(1, sequence_length, input_data_lstm.shape[1]),
                                    input_data_tcn.reshape(1, sequence_length, input_data_tcn.shape[1])])
        predictions.append(prediction[0, 0])

        # Shift the input data by one time step and append the new prediction
        input_data_lstm = np.roll(input_data_lstm, shift=-1, axis=0)
        input_data_lstm[-1, -1] = prediction[0, 0]

        input_data_tcn = np.roll(input_data_tcn, shift=-1, axis=0)
        input_data_tcn[-1, -1] = prediction[0, 0]

    return predictions


# Day

In [7]:
# Prepare data with daily intervals
data_day = prepare_data(df_selected, time_intervals['day'])

# Create sequences and targets
sequences_day, targets_day = create_sequences(data_day, sequence_length)

# Split the data into training and testing sets
X_train_day, X_test_day, y_train_day, y_test_day = train_test_split(sequences_day, targets_day, test_size=0.2, random_state=1)

# Build the LSTM model
lstm_model_day = Sequential()
lstm_model_day.add(LSTM(50, input_shape=(X_train_day.shape[1], X_train_day.shape[2])))
lstm_model_day.add(Dense(1, activation='sigmoid'))
lstm_model_day.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
lstm_model_day.fit(X_train_day, y_train_day, epochs=10, batch_size=16, validation_split=0.15)

# Build the TCN model
tcn_model_day = Sequential([
    TCN(input_shape=(sequence_length, X_train_day.shape[2])),
    Dense(1, activation='sigmoid')
])

# Compile the model
tcn_model_day.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
tcn_model_day.fit(X_train_day, y_train_day, epochs=20, batch_size=32, validation_split=0.1)

# Function to create a hybrid model with attention mechanism
def create_attention_hybrid_model(lstm_model, tcn_model):
    lstm_input = lstm_model.input
    tcn_input = tcn_model.input

    # Get the output layers of both models
    lstm_output = lstm_model.layers[-1].output
    tcn_output = tcn_model.layers[-1].output

    # Use Attention mechanism to combine outputs
    attention = Attention()([lstm_output, tcn_output])
    merged = Concatenate()([lstm_output, tcn_output, attention])

    # Add a dense layer for the final prediction
    merged = Dense(1, activation='sigmoid')(merged)

    # Create the ensemble model
    ensemble_model = Model(inputs=[lstm_input, tcn_input], outputs=merged)

    # Compile the model
    ensemble_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

    return ensemble_model

# Select a starting point for predictions
input_data_lstm_hybrid = X_test_day[3]
input_data_tcn_hybrid = X_test_day[3]

# Number of time steps to predict into the future
prediction_steps_hybrid = 7

# Create the hybrid model
hybrid_model_day_attention = create_attention_hybrid_model(lstm_model_day, tcn_model_day)

# Train the hybrid model with both LSTM and TCN data
hybrid_model_day_attention.fit([X_train_day, X_train_day], y_train_day, epochs=20, batch_size=32, validation_split=0.1)


Epoch 1/10
4/4 [==============================] - 3s 206ms/step - loss: 0.3050 - mean_squared_error: 0.3050 - val_loss: 0.2726 - val_mean_squared_error: 0.2726
Epoch 2/10
4/4 [==============================] - 0s 33ms/step - loss: 0.2713 - mean_squared_error: 0.2713 - val_loss: 0.2297 - val_mean_squared_error: 0.2297
Epoch 3/10
4/4 [==============================] - 0s 29ms/step - loss: 0.2432 - mean_squared_error: 0.2432 - val_loss: 0.2124 - val_mean_squared_error: 0.2124
Epoch 4/10
4/4 [==============================] - 0s 27ms/step - loss: 0.2064 - mean_squared_error: 0.2064 - val_loss: 0.1574 - val_mean_squared_error: 0.1574
Epoch 5/10
4/4 [==============================] - 0s 33ms/step - loss: 0.1876 - mean_squared_error: 0.1876 - val_loss: 0.1351 - val_mean_squared_error: 0.1351
Epoch 6/10
4/4 [==============================] - 0s 29ms/step - loss: 0.1703 - mean_squared_error: 0.1703 - val_loss: 0.1123 - val_mean_squared_error: 0.1123
Epoch 7/10
4/4 [=============================

In [8]:
# Evaluate the model using Mean Squared Error
mse_day_attention = hybrid_model_day_attention.evaluate([X_test_day, X_test_day], y_test_day, verbose=0)[1]
print(f'Model Mean Squared Error: {mse_day_attention:.4f}\n')

# Make predictions with the hybrid model
predicted_failures_hybrid_attention = predict_future_failures_hybrid(hybrid_model_day_attention, input_data_lstm_hybrid, input_data_tcn_hybrid, sequence_length, prediction_steps_hybrid)

# Print the predicted failures
print("Predicted Failures for the Next 7 Time Steps (Hybrid):")
print(predicted_failures_hybrid_attention)

# Evaluate the predictions using Mean Squared Error
mse_predictions = np.mean((predicted_failures_hybrid_attention - y_test_day[3:3+prediction_steps_hybrid])**2)
print(f'\nMean Squared Error for Predictions: {mse_predictions:.4f}\n')

# Evaluate the predictions using Mean Absolute Error
mae_predictions_hybrid_attention_day = np.mean(np.abs(predicted_failures_hybrid_attention - y_test_day[3:3+prediction_steps_hybrid]))
print(f'Mean Absolute Error for Predictions: {mae_predictions_hybrid_attention_day:.4f}')


Model Mean Squared Error: 0.1476
1/1 [==============================] - 0s 25ms/step
Predicted Failures for the Next 7 Time Steps (Hybrid):
[0.64054656, 0.66435, 0.67517656, 0.6723426, 0.678719, 0.6767012, 0.66627884]
Mean Absolute Error for Predictions: 0.3323
Mean Squared Error for Predictions: 0.1106


# Hour

In [12]:
# Prepare data with hourly intervals
data_hour = prepare_data(df_selected, time_intervals['hour'])

# Create sequences and targets
sequences_hour, targets_hour = create_sequences(data_hour, sequence_length)

# Split the data into training and testing sets
X_train_hour, X_test_hour, y_train_hour, y_test_hour = train_test_split(sequences_hour, targets_hour, test_size=0.2, random_state=1)

# Build the LSTM model
lstm_model_hour = Sequential()
lstm_model_hour.add(LSTM(50, input_shape=(X_train_hour.shape[1], X_train_hour.shape[2])))
lstm_model_hour.add(Dense(1, activation='sigmoid'))
lstm_model_hour.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
lstm_model_hour.fit(X_train_hour, y_train_hour, epochs=10, batch_size=16, validation_split=0.15)

# Build the TCN model
tcn_model_hour = Sequential([
    TCN(input_shape=(sequence_length, X_train_hour.shape[2])),
    Dense(1, activation='sigmoid')
])

# Compile the model
tcn_model_hour.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
tcn_model_hour.fit(X_train_hour, y_train_hour, epochs=20, batch_size=32, validation_split=0.1)

# Function to create a hybrid model with attention mechanism
def create_attention_hybrid_model(lstm_model, tcn_model):
    lstm_input = lstm_model.input
    tcn_input = tcn_model.input

    # Get the output layers of both models
    lstm_output = lstm_model.layers[-1].output
    tcn_output = tcn_model.layers[-1].output

    # Use Attention mechanism to combine outputs
    attention = Attention()([lstm_output, tcn_output])
    merged = Concatenate()([lstm_output, tcn_output, attention])

    # Add a dense layer for the final prediction
    merged = Dense(1, activation='sigmoid')(merged)

    # Create the ensemble model
    ensemble_model = Model(inputs=[lstm_input, tcn_input], outputs=merged)

    # Compile the model
    ensemble_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

    return ensemble_model

# Select a starting point for predictions
input_data_lstm_hybrid = X_test_hour[3]
input_data_tcn_hybrid = X_test_hour[3]

# Number of time steps to predict into the future
prediction_steps_hybrid = 7

# Create the hybrid model
hybrid_model_hour_attention = create_attention_hybrid_model(lstm_model_hour, tcn_model_hour)

# Train the hybrid model with both LSTM and TCN data
hybrid_model_hour_attention.fit([X_train_hour, X_train_hour], y_train_hour, epochs=20, batch_size=32, validation_split=0.1)


Epoch 1/10
116/116 [==============================] - 5s 25ms/step - loss: 0.1794 - mean_squared_error: 0.1794 - val_loss: 0.1474 - val_mean_squared_error: 0.1474
Epoch 2/10
116/116 [==============================] - 3s 22ms/step - loss: 0.1381 - mean_squared_error: 0.1381 - val_loss: 0.1343 - val_mean_squared_error: 0.1343
Epoch 3/10
116/116 [==============================] - 3s 22ms/step - loss: 0.1370 - mean_squared_error: 0.1370 - val_loss: 0.1258 - val_mean_squared_error: 0.1258
Epoch 4/10
116/116 [==============================] - 3s 22ms/step - loss: 0.1162 - mean_squared_error: 0.1162 - val_loss: 0.1132 - val_mean_squared_error: 0.1132
Epoch 5/10
116/116 [==============================] - 3s 22ms/step - loss: 0.1056 - mean_squared_error: 0.1056 - val_loss: 0.1039 - val_mean_squared_error: 0.1039
Epoch 6/10
116/116 [==============================] - 3s 22ms/step - loss: 0.1029 - mean_squared_error: 0.1029 - val_loss: 0.1001 - val_mean_squared_error: 0.1001
Epoch 7/10
116/116 [==

In [13]:
# Evaluate the model using Mean Squared Error
mse_hour_attention = hybrid_model_hour_attention.evaluate([X_test_hour, X_test_hour], y_test_hour, verbose=0)[1]
print(f'Model Mean Squared Error: {mse_hour_attention:.4f}\n')

# Make predictions with the hybrid model
predicted_failures_hybrid_attention = predict_future_failures_hybrid(hybrid_model_hour_attention, input_data_lstm_hybrid, input_data_tcn_hybrid, sequence_length, prediction_steps_hybrid)

# Print the predicted failures
print("Predicted Failures for the Next 7 Time Steps (Hybrid):")
print(predicted_failures_hybrid_attention)

# Evaluate the predictions using Mean Squared Error
mse_predictions = np.mean((predicted_failures_hybrid_attention - y_test_hour[3:3+prediction_steps_hybrid])**2)
print(f'\nMean Squared Error for Predictions: {mse_predictions:.4f}\n')

# Evaluate the predictions using Mean Absolute Error
mae_predictions_hybrid_attention_hour = np.mean(np.abs(predicted_failures_hybrid_attention - y_test_hour[3:3+prediction_steps_hybrid]))
print(f'Mean Absolute Error for Predictions: {mae_predictions_hybrid_attention_hour:.4f}\n')


Model Mean Squared Error: 0.1275

1/1 [==============================] - 0s 23ms/step
Predicted Failures for the Next 7 Time Steps (Hybrid):
[0.3634331, 0.3227131, 0.3204667, 0.3197397, 0.31916603, 0.31874532, 0.31914416]
Mean Absolute Error for Predictions: 0.4169

Mean Squared Error for Predictions: 0.1973


# Minute

In [29]:
# Prepare data
data_minute = prepare_data(df_selected, time_intervals['minute'])

# Create sequences and targets
sequences_minute, targets_minute = create_sequences(data_minute, sequence_length)

# Split the data into training and testing sets
X_train_minute, X_test_minute, y_train_minute, y_test_minute = train_test_split(sequences_minute, targets_minute, test_size=0.2, random_state=1)

# Build the LSTM model
lstm_model_minute = Sequential()
lstm_model_minute.add(LSTM(50, input_shape=(X_train_minute.shape[1], X_train_minute.shape[2])))
lstm_model_minute.add(Dense(1, activation='sigmoid'))
lstm_model_minute.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
lstm_model_minute.fit(X_train_minute, y_train_minute, epochs=10, batch_size=16, validation_split=0.15)

# Build the TCN model
tcn_model_minute = Sequential([
    TCN(input_shape=(sequence_length, X_train_minute.shape[2])),
    Dense(1, activation='sigmoid')
])

# Compile the model
tcn_model_minute.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
tcn_model_minute.fit(X_train_minute, y_train_minute, epochs=20, batch_size=32, validation_split=0.1)

# Function to create a hybrid model with attention mechanism
def create_attention_hybrid_model(lstm_model, tcn_model):
    lstm_input = lstm_model.input
    tcn_input = tcn_model.input

    # Get the output layers of both models
    lstm_output = lstm_model.layers[-1].output
    tcn_output = tcn_model.layers[-1].output

    # Use Attention mechanism to combine outputs
    attention = Attention()([lstm_output, tcn_output])
    merged = Concatenate()([lstm_output, tcn_output, attention])

    # Add a dense layer for the final prediction
    merged = Dense(1, activation='sigmoid')(merged)

    # Create the ensemble model
    ensemble_model = Model(inputs=[lstm_input, tcn_input], outputs=merged)

    # Compile the model
    ensemble_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

    return ensemble_model

# Select a starting point for predictions
input_data_lstm_hybrid = X_test_minute[3]
input_data_tcn_hybrid = X_test_minute[3]

# Number of time steps to predict into the future
prediction_steps_hybrid = 7

# Create the hybrid model
hybrid_model_minute_attention = create_attention_hybrid_model(lstm_model_minute, tcn_model_minute)

# Train the hybrid model with both LSTM and TCN data
hybrid_model_minute_attention.fit([X_train_minute, X_train_minute], y_train_minute, epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10
 516/7033 [=>............................] - ETA: 2:13 - loss: 0.0309 - mean_squared_error: 0.0309

KeyboardInterrupt: 

In [ ]:
# Evaluate the model using Mean Squared Error
mse_minute_attention = hybrid_model_minute_attention.evaluate([X_test_minute, X_test_minute], y_test_minute, verbose=0)[1]
print(f'Model Mean Squared Error: {mse_minute_attention:.4f}\n')

# Make predictions with the hybrid model
predicted_failures_hybrid_attention = predict_future_failures_hybrid(hybrid_model_minute_attention, input_data_lstm_hybrid, input_data_tcn_hybrid, sequence_length, prediction_steps_hybrid)

# Print the predicted failures
print("Predicted Failures for the Next 7 Time Steps (Hybrid):")
print(predicted_failures_hybrid_attention)

# Evaluate the predictions using Mean Squared Error
mse_predictions = np.mean((predicted_failures_hybrid_attention - y_test_minute[3:3+prediction_steps_hybrid])**2)
print(f'\nMean Squared Error for Predictions: {mse_predictions:.4f}\n')

# Evaluate the predictions using Mean Absolute Error
mae_predictions_hybrid_attention_minute = np.mean(np.abs(predicted_failures_hybrid_attention - y_test_minute[3:3+prediction_steps_hybrid]))
print(f'Mean Absolute Error for Predictions: {mae_predictions_hybrid_attention_minute:.4f}\n')


Model Mean Squared Error: 0.0013

1/1 [==============================] - 0s 22ms/step
Predicted Failures for the Next 7 Time Steps (Hybrid):
[0.0026827534, 0.0026833082, 0.0026832204, 0.0026832472, 0.0026831846, 0.0026832088, 0.0026832088]

Mean Squared Error for Predictions: 0.4263

Mean Absolute Error for Predictions: 0.4290

